### Goal: Convex_Boundary_Inter_Distance
**Inputs:**
1. G1 = N1 x 2 matrix; each row is a point in group G1
2. G2 = N2 x 2 matrix; each row is a point in group G2
3. ratio = the ratio in the function "Boundary"
    
**Output:** 
a dictionary with 3 keys ("Group1", "Group2", "D_min"), whose values are the points in each group achieving the minimum distance and the so-computed minimum distance.


In [1]:
Pts = rand(50,2)
Centers = rand(5,2)
using Plots
plotly()
scatter(Pts[:,1], Pts[:,2], label = "Points")
scatter!(Centers[:,1], Centers[:,2], label = "Centers")

In [3]:
using QHull

p = rand(6000,2)
@time ch = chull(p)
ch.points         # original points
ch.vertices       # indices to line segments forming the convex hull
ch.simplices;      # the simplexes forming the convex hull
## show(ch)

  0.005709 seconds (1.35 k allocations: 135.000 KiB)


In [26]:
normal_b

2-element Array{Float64,1}:
 -0.9865478606340647
  0.7944371599677893

In [46]:
Bds = ch.simplices
plot()
for b = 1:length(Bds)
    plot!(p[Bds[b],:][:,1], p[Bds[b],:][:,2])
end
plot!()

In [164]:
using LinearAlgebra ## for "dot"
ratio = 1/50
## plot for illustration
## scatter(p[:,1], p[:,2], markersize = 1, label = "points")
## scatter!(p[ch.vertices,1], p[ch.vertices,2], label = "vertices")
Bds = ch.simplices 
Bd_Pts = Int[]
for b = 1:length(Bds)
    vector_b = p[Bds[b][1],:]-p[Bds[b][2],:] ## the vector of the boundary b
    normal_b = [-vector_b[2], vector_b[1]] ## a vector normal to the boundary b vector
    ## move vector_b orthogonally and find out the intersected bar
    bd_b_para = p*vector_b 
    Range_b_para = p[ch.simplices[b],:]*vector_b
    pts_b_para = findall((p*vector_b.<=maximum(Range_b_para)).&(minimum(Range_b_para).<=p*vector_b))
    N = Int(round(ratio*length(pts_b_para)))
    Values_Labels = sortslices([p[pts_b_para,:]*normal_b pts_b_para], dims = 1)
    Values = Values_Labels[:,1]
    Labels = Values_Labels[:,2]
    Bd_Pts_N = 0
    if abs((Values[1]-Values[2])/Values[2])<1.0e-5
        Bd_Pts_N = Array{Int,1}(Labels[1:N])
    else
        Bd_Pts_N = Array{Int,1}(Labels[length(Labels)-N+1:length(Labels)])
    end
    Bd_Pts = union(Bd_Pts, Bd_Pts_N)
    ## plot for sanity check
    ## plot!(p[Bds[b],:][:,1], p[Bds[b],:][:,2], label = "Bd $b")
    ## scatter!(p[Bd_Pts_N,1], p[Bd_Pts_N,2], label = "Bd $b points")
end
## scatter!()

In [170]:
G1 = p[Bd_Pts,:]

447×2 Array{Float64,2}:
 0.998571  0.204133  
 0.994234  0.00768586
 0.998208  0.192265  
 0.994276  0.0380312 
 0.99207   0.0199779 
 0.990908  0.0131281 
 0.991446  0.0532527 
 0.993894  0.176328  
 0.989554  0.0696217 
 0.988762  0.0626323 
 0.986338  0.012406  
 0.988483  0.122136  
 0.985169  0.013992  
 ⋮                   
 0.378143  0.994879  
 0.393624  0.99496   
 0.365081  0.995066  
 0.41956   0.996673  
 0.387986  0.997431  
 0.451204  0.997506  
 0.324915  0.997679  
 0.452715  0.99804   
 0.344065  0.998802  
 0.32475   0.999008  
 0.393849  0.999023  
 0.350226  0.999346  

In [181]:
?find

search: findmin findmax findall findprev findnext findmin! findmax! findlast

Couldn't find find
Perhaps you meant bind, sind, fill, font, fd, fld, cond, pinv, Cint, asind or in


No documentation found.

Binding `find` does not exist.


In [189]:
G1 = rand(500,2)
G2 = rand(600,2)
@time Dist2 = [sum((G1[i,:]-G2[j,:]).^2) for i=1:size(G1,1), j = 1:size(G2,1)] ## distance^2
@time G1_G2_Min = findmin(Dist2)
Min_Index = G1_G2_Min[2]
Min_Dist = sqrt(G1_G2_Min[1])

  1.013138 seconds (3.97 M allocations: 166.378 MiB, 3.55% gc time)
  0.001162 seconds (5 allocations: 192 bytes)


0.00045287259214233364

In [31]:
using LinearAlgebra ## "dot"
using QHull ## "chull"
## using Plots
## plotly()

function Boundary(G::Array{Float64,2}, ratio = 0.05::Float64) ## G = Group (of points), N x 2
    @time ch = chull(G)
    ## ch.points         # original points
    ## ch.vertices       # indices to line segments forming the convex hull
    ## ch.simplices;      # the simplexes forming the convex hull
    ## show(ch) 
        
        
    ## ratio = 1/50
    ## plot for illustration
    ## scatter(G[:,1], G[:,2], markersize = 1, label = "points")
    ## scatter!(G[ch.vertices,1], G[ch.vertices,2], label = "vertices")
    Bds = ch.simplices 
    Bd_Pts = Int[]
    for b = 1:length(Bds)
        vector_b = G[Bds[b][1],:]-G[Bds[b][2],:] ## the vector of the boundary b
        normal_b = [-vector_b[2], vector_b[1]] ## a vector normal to the boundary b vector
        ## move vector_b orthogonally and find out the intersected bar
        bd_b_para = G*vector_b 
        Range_b_para = G[ch.simplices[b],:]*vector_b
        pts_b_para = findall((G*vector_b.<=maximum(Range_b_para)).&(minimum(Range_b_para).<=G*vector_b))
        N = Int(round(ratio*length(pts_b_para)))
        Values_Labels = sortslices([G[pts_b_para,:]*normal_b pts_b_para], dims = 1)
        Values = Values_Labels[:,1]
        Labels = Values_Labels[:,2]
        Bd_Pts_N = 0
        if abs((Values[1]-Values[2])/Values[2])<1.0e-5
            Bd_Pts_N = Array{Int,1}(Labels[1:N])
        else
            Bd_Pts_N = Array{Int,1}(Labels[length(Labels)-N+1:length(Labels)])
        end
        Bd_Pts = union(Bd_Pts, Bd_Pts_N)
        ## plot for sanity check
        ## plot!(G[Bds[b],:][:,1], G[Bds[b],:][:,2], label = "Bd $b")
        ## scatter!(G[Bd_Pts_N,1], G[Bd_Pts_N,2], label = "Bd $b points")
    end
    ## scatter!()
    Bd_Pts
end

Boundary (generic function with 2 methods)

In [21]:
GG = rand(1000,2)
Boundary(GG)

  0.004820 seconds (995 allocations: 46.281 KiB)


172-element Array{Int64,1}:
 188
 943
 905
 987
 926
 566
 930
 193
 399
 698
 725
 889
 842
   ⋮
  88
 859
 166
  51
 326
 231
 886
 654
 668
 914
  25
 894

In [20]:
## computes by brute force the minimum pairwise distances between two groups of points
## Input: G1, G2, N x d
## Output: 
## (1) the pair of points in G1 and G2, resp, achieving the minimum
## (2) the minimum distance
function Inter_Distance(G1::Array{Float64,2}, G2::Array{Float64,2})
    @time Dist2 = [sum((G1[i,:]-G2[j,:]).^2) for i=1:size(G1,1), j = 1:size(G2,1)] ## distance^2
    @time G1_G2_Min = findmin(Dist2)
    ## Min_Index = G1_G2_Min[2]
    ## Min_Dist = sqrt(G1_G2_Min[1])
    return (G1_G2_Min[2], sqrt(G1_G2_Min[1]))
end

Inter_Distance (generic function with 1 method)

In [22]:
## This function computes an approximate minimum distance bewteen two groups of points
## Inputs: 
## (1) G1 = N1 x 2 matrix; each row is a point in group G1
## (2) G2 = N2 x 2 matrix; each row is a point in group G2
## (3) ratio = the ratio in the function "Boundary"
## Output:
## a dictionary with 3 keys ("Group1", "Group2", "D_min")
## whose values are the points in each group achieving the minimum distance 
## and the so-computed minimum distance.
function Convex_Boundary_Inter_Distance(G1::Array{Float64,2}, G2::Array{Float64,2}, ratio = 0.05::Float64)
    Bd1_Label = Boundary(G1, ratio)
    Bd2_Label = Boundary(G2, ratio)
    G1_Bd1 = G1[Bd1_Label,:]
    G2_Bd2 = G2[Bd2_Label,:]
    (NewLabel, Dist_Min) = Inter_Distance(G1_Bd1, G2_Bd2)
    return Dict("Group1"=>Bd1_Label[NewLabel[1]], "Group2"=>Bd2_Label[NewLabel[2]], "D_min"=>Dist_Min)
end

Convex_Boundary_Inter_Distance (generic function with 1 method)

## Test the function

In [38]:
H1 = rand(300, 2)
H2 = rand(500, 2).+[1 1]
scatter(H1[:,1], H1[:,2], label = "Group 1")
scatter!(H2[:,1], H2[:,2], label = "Group 2")
CBID = Convex_Boundary_Inter_Distance(H1,H2)

  0.002909 seconds (753 allocations: 28.484 KiB)
  0.002689 seconds (711 allocations: 30.375 KiB)
  0.004248 seconds (18.23 k allocations: 1.474 MiB)
  0.000027 seconds


Dict{String,Real} with 3 entries:
  "Group2" => 88
  "Group1" => 7
  "D_min"  => 0.0848971

In [39]:
scatter!([H1[CBID["Group1"],1]], [H1[CBID["Group1"],2]], label = "Point for Group 1")
scatter!([H2[CBID["Group2"],1]], [H2[CBID["Group2"],2]], label = "Point for Group 2")

In [2]:
include("src\\Convex_Boundary_Inter_Distance.jl")

┌ Info: Recompiling stale cache file C:\Users\wmg97_000\.julia\compiled\v1.0\QHull\FsLaA.ji for QHull [a8468747-bd6f-53ef-9e5c-744dbc5c59e7]
└ @ Base loading.jl:1190


Convex_Boundary_Inter_Distance (generic function with 2 methods)

In [3]:
Inter_Distance

Inter_Distance (generic function with 1 method)

In [3]:
Boundary

Boundary (generic function with 2 methods)

In [4]:
Convex_Boundary_Inter_Distance

Convex_Boundary_Inter_Distance (generic function with 2 methods)

In [7]:
H1 = rand(10000, 2)
H2 = rand(10000, 2).+[1 1];

In [8]:
CBID = @time Convex_Boundary_Inter_Distance(H1,H2)

  2.210439 seconds (14.59 M allocations: 1.173 GiB, 10.08% gc time)


Dict{String,Real} with 3 entries:
  "Group2" => 10000
  "Group1" => 6458
  "D_min"  => 0.00869799

In [20]:
@time Inter_Distance(H1,H2)

  7.884744 seconds (75.00 M allocations: 5.923 GiB, 9.11% gc time)
  0.063881 seconds
  7.950199 seconds (75.00 M allocations: 5.923 GiB, 9.03% gc time)


(CartesianIndex(2267, 3911), 0.03344129473255766)